In [ ]:
import os 
import sys
from pathlib import Path

sys.path.append(str(Path('/home/raincake/expense-telegram-bot').resolve()))

from src.settings import BASE_DIR, TOKEN
from telegram.ext import ApplicationBuilder

In [32]:
log_cfg_path

PosixPath('/home/raincake/expense-telegram-bot/config/logging.conf')

In [ ]:
import logging 
# Definimos el logging para tener claro los logs y eso del bot:
log_cfg_path = BASE_DIR / "config" / "logging.conf"
logging.config.fileConfig(log_cfg_path, disable_existing_loggers=False)

# A partir de aquí puedo definir distintos loggers (objetos que permiten registrar
# mensajes y eventos durante la ejecución de una app, el logger se encarga de gestionarlos,
# los mensajes se envían a un handler que puede ser un archivo, la consola, etc.)
logger = logging.getLogger("expense_bot")

In [57]:
async def star_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = update.effective_user.id
    user = update.effective_user.first_name
    await update.message.reply_text(
            f"Hola {user_id}, {user}"
        )

In [58]:
def main() -> None:
    """
    Función principal de la ejecución del bot
    """

    logger.info("Iniciando el Bot...")
    application = ApplicationBuilder().token(TOKEN).build()
    start_handler = CommandHandler('start', star_command)
    # Aquí añadimos los distintos handlers ...
    application.add_handler(start_handler)

    logger.info("Bot iniciado, esperando los mensajes...")

    application.run_polling()


In [55]:
import nest_asyncio
nest_asyncio.apply()

In [59]:
main()

INFO Iniciando el Bot...
INFO Bot iniciado, esperando los mensajes...


RuntimeError: Cannot close a running event loop

In [ ]:
from src.utils.user_utils import load_users